In [1]:
%%time
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 


from Artha.nlp_extraction import *
import Artha.data_process as dp
from Artha.neocoinsdb import Neo
import Artha.crypto_data as crypto
from tqdm import tqdm
import csv
import json
from collections import Counter
import numpy as np
np.set_printoptions(suppress=True)
import pprint
import math
from datetime import datetime

pp = pprint.PrettyPrinter(indent=1)

# coins database
neo = Neo("neo4j://localhost:7687", "neo4j", "part2")

In [2]:
print(len(neo.get_nodes()), len(neo.get_relations()))

402 0


In [3]:
neo.clear_db()

'Deleted'

In [4]:
neo.load_coins_data()

Updated coins


In [6]:
%%time
# username = "BTC_JackSparrow"
# username = "razoreth"
# username = "Nostranomist"

username = "CryptoKaleo"
docs = run_pipeline(username)
mentions = get_mention_scores(username, docs)

CPU times: user 2.53 s, sys: 164 ms, total: 2.69 s
Wall time: 4.75 s


In [8]:
%%time

mentions = np.zeros((len(tick_dict), len(docs)))

# fill in values of dict lists where ticker is mentioned
for ind, doc in tqdm(enumerate(docs)):
    for tick in doc._.tickers:
        mentions[tick_dict[tick]][ind] += 1

# function of array of tweet times
tweet_times = db.expo_func(np.array([dp.time_diff(doc._.tweeted_at) for doc in docs]).reshape((len(docs),1)))

# multiply matrix of mentions by time vector
mul = np.matmul(mentions, tweet_times).round(5).reshape(1, len(all_ticks))[0]

# list of edges to add
mention_edges = list(zip([username] * len(docs), list(tick_dict.keys()), mul))

# write edges
with open('mentions1.csv','w+') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['username', 'coin', 'weight'])
    for row in mention_edges:
        csv_out.writerow(row)

1157it [00:00, 28701.77it/s]CPU times: user 67 ms, sys: 4.24 ms, total: 71.3 ms
Wall time: 70.9 ms

